In [1]:
##########################################################################
# 인스타 그램의 사진과 해쉬태그 수집하기 
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import math
import os
import random
import unicodedata       # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈
import urllib.request
import urllib
import pandas as pd

#Step 2. 사용자에게 필요한 정보들을 입력 받기
print("=" *80)
print("인스타그램 해쉬태그와 이미지 수집하기")
print("=" *80)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
try :
    cnt = int( input('4.수집할 건수는 총 몇건입니까?(기본값:10): '))
except ValueError :
    cnt = 10
    print('기본값인 10 건으로 수집을 진행합니다.')
page_cnt = math.ceil( cnt / 10)
f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\py_temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\py_temp\\"

인스타그램 해쉬태그와 이미지 수집하기
1.인스타그램의 ID를 입력하세요: jmj0528.hound
2.인스타그램의 비밀번호를 입력하세요: mhjmhj0122!
3.검색할 해쉬태그를 입력하세요(예: 강남맛집): 청주
4.수집할 건수는 총 몇건입니까?(기본값:10): 2
5.파일이 저장될 경로만 쓰세요(기본경로 : c:\py_temp\ ) : 


In [2]:
#Step 3.결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성하기
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

sec_name='인스타그램'
img_dir = f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+'image'

os.makedirs(img_dir)
os.chdir(img_dir)

fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xlsx'

# Step 4. 인스타그램 접속 후 자동 로그인 하기
s_time = time.time( )
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)
url = "https://www.instagram.com/"
driver.get(url)
time.sleep(random.randrange(1,5))

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

#ID와 비번 입력후 로그인하기
eid = driver.find_element(By.NAME,'username')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element(By.NAME,'password')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)
    
driver.find_element(By.XPATH,'//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(5)



요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^




In [3]:
# Step 5. 검색할 해쉬태그 입력하기
element = driver.find_element(By.XPATH,'//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)
    
driver.find_element(By.XPATH,'//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div').click()
time.sleep(5)

# Step 6. 전체 게시물의 원본 URL 추출하기
item=[]     # 인스타그램 URL 주소 저장할 리스트
item2=[]    # 중복값을 제거한 최종 URL 주소를 저장할 리스트

# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(5)

print('요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^')
print()

a = 1
while (a <= page_cnt):
    scroll_down(driver) 
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    all_a = soup.find('article','KC1QD').find_all('a')

    for i in all_a:    
        url = i['href']
        item.append(url)       
        item2 = pd.Series(item).drop_duplicates()
    
        if len(item2) >= cnt :
            break
    a += 1
    print('요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^')

# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 1
print('= 수집될 인스타그램 주소는 아래와 같습니다 =========')
for x in item2 :
    url = 'https://www.instagram.com' + x
    full_url.append(url)
    print(url_cnt,':',url)
    
    if url_cnt > cnt:
        break
    url_cnt += 1
print('====================================================')
print()

#Step 7. 각 페이지별로 이미지와 해쉬태그를 수집하기
count = 1     # 추출 데이터 건수 세기
no2= []       # 번호 저장
url2=[]       # 수집완료된 url 저장
hash2 = []    # 해쉬 태그 저장 

count = 1

for c in full_url :
    print()
    driver.get(c)
    time.sleep(random.randrange(3,9))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.find('div','EtaWk')

    try :
        tags_1 = tags.find_all('a')
    except :
        continue
    else :
        print('%s번째 게시물의 대표 이미지와 해쉬태그를 수집합니다~~~' %count)
        print('게시물 URL:' , c )
        no2.append(count)
        url2.append(c)
        
        #해당 페이지의 대표 이미지 수집
        img_src = soup.find('div','KL4Bh').find('img')['src']
        urllib.request.urlretrieve(img_src , str(count)+'.jpg')
        print(img_dir,'아래에 %s번째 이미지 저장 완료===' %count)

        # 해당 페이지의 해시태그 수집
        # 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만들기
        import sys
        bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

        hash_tags=[]
        for d in tags_1 :
            tags = d.get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)
                      
            if tags_2[0:1]=='#':
                
                hash_tags.append(tags_2)
                
        print(hash_tags)       
        hash2.append(hash_tags)  # 각 게시물의 해시태그를 1줄의 리스트 형태로 저장하기
        
    count += 1
    

#Step 8. 수집된 해시태그를 csv , xls 형식으로 저장하기
# xls , csv로 저장하기 위해 데이터 프레임 생성하기
insta = pd.DataFrame( )
insta['번호'] = no2
insta['URL주소'] = url2
insta['해쉬태그'] = pd.Series(hash2)

# csv 형태로 저장하기
insta.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
insta.to_excel(fx_name ,index=False , engine='openpyxl')

#Step 9. 요약 정보 출력하기    
e_time = time.time( )
t_time = e_time - s_time

print("=" *120)
print("1.총 소요시간: %s 초" %round(t_time,1))
print("2.총 저장 건수: %s 건 " %count)
print("3.csv파일 저장 경로: %s" %fc_name)
print("4.xls파일 저장 경로: %s" %fx_name)
print("5.이미지파일 저장 경로: %s" %img_dir)
print("=" *120)

driver.close( )

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x005D6463+2188387]
	Ordinal0 [0x0056E461+1762401]
	Ordinal0 [0x00483D78+802168]
	Ordinal0 [0x004B1880+989312]
	Ordinal0 [0x004B1B1B+989979]
	Ordinal0 [0x004DE912+1173778]
	Ordinal0 [0x004CC824+1099812]
	Ordinal0 [0x004DCC22+1166370]
	Ordinal0 [0x004CC5F6+1099254]
	Ordinal0 [0x004A6BE0+945120]
	Ordinal0 [0x004A7AD6+948950]
	GetHandleVerifier [0x008771F2+2712546]
	GetHandleVerifier [0x0086886D+2652765]
	GetHandleVerifier [0x0066002A+520730]
	GetHandleVerifier [0x0065EE06+516086]
	Ordinal0 [0x0057468B+1787531]
	Ordinal0 [0x00578E88+1805960]
	Ordinal0 [0x00578F75+1806197]
	Ordinal0 [0x00581DF1+1842673]
	BaseThreadInitThunk [0x757CFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77947A9E+286]
	RtlGetAppContainerNamedObjectP